In [1]:
from google.cloud import storage
%cd -q /home/dataproc
!ls inverted_index_gcp.py

#!mkdir text title anchor search_text search_title
from inverted_index_gcp import InvertedIndex

bucket_name = 'ir-project-noam-and-shiri'

inverted_index_gcp.py


In [2]:
import nltk
nltk.download('stopwords')
import search_backend as sb
from BM25_from_index import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

# index_dst = "index_text"+".pkl"
# index_src = f'gs://{bucket_name}/text/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_text"+'_dct'+".pkl"
# index_src = f'gs://{bucket_name}/text/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_anchor"+".pkl"
# index_src = f'gs://{bucket_name}/anchor/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_anchor"+'_dct'+".pkl"
# index_src = f'gs://{bucket_name}/anchor/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_title"+".pkl"
# index_src = f'gs://{bucket_name}/title/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_title"+'_dct'+".pkl"
# index_src = f'gs://{bucket_name}/title/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

# index_dst = "index_title"+'_names'+".pkl"
# index_src = f'gs://{bucket_name}/title/{index_dst}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst
# print("finished")



In [4]:
index_dst = "pagerank"+".pkl"
index_src = f'gs://{bucket_name}/pagerank/{index_dst}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst
print("finished")

Copying gs://ir-project-noam-and-shiri/pagerank/pagerank.pkl...
\ [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     
CommandException: "ls" command does not support "file://" URLs. Did you mean to use a gs:// URL?
finished


In [5]:
idx_title = InvertedIndex.read_index('.', 'index_title')
idx_text = InvertedIndex.read_index('.', 'index_text')
idx_anchor = InvertedIndex.read_index('.', 'index_anchor')
dct_title = InvertedIndex.read_index('.', 'index_title_names')
DL_title =  InvertedIndex.read_index('.', 'index_title_dct')
DL_text = InvertedIndex.read_index('.', 'index_text_dct')
DL_anchor = InvertedIndex.read_index('.', 'index_anchor_dct')
print("before BM25_from_index")

before BM25_from_index


In [14]:

def search_bm25_improved(clean_query):
    bm25title = BM25_from_index(idx_title,DL_title)
    bm25text = BM25_from_index(idx_text,DL_text)
    bm25anchor = BM25_from_index(idx_anchor,DL_anchor)
    title_candidates = bm25title.search(clean_query,bucket_name,'title')
    text_candidates = bm25text.search(clean_query,bucket_name,'text')
    anchor_candidates = bm25anchor.search(clean_query,bucket_name,'anchor')
    best_text_title = merge_results(title_candidates, text_candidates , anchor_candidates , title_weight=0.1,text_weight=0.5, anchor_weight=0.2,pagerank_weight=0.2, N=100)
    return list(map(lambda tup: (int(tup[0]), dct_title[tup[0]]) , best_text_title ))


def merge_results(title_scores,body_scores,anchor_scores,title_weight=0.25,text_weight=0.25,anchor_weight=0.25,pagerank_weight=0.25,N = 3):    
    # YOUR CODE HERE
    title_dict = dict(title_scores)
    body_dict = dict(body_scores)
    anchor_dict = dict(anchor_scores)
    docs_id_for_query = set(list(title_dict.keys())+list(body_dict.keys())+list(anchor_dict.keys()))
    allscores = []
    i=0
    pagerank = sb.get_page_stats(docs_id_for_query, "./pagerank.pkl")
    for docid in docs_id_for_query:
        allscores.append((docid,title_dict.get(docid,0)*title_weight + body_dict.get(docid,0)*text_weight - anchor_dict.get(docid,0)*anchor_weight + pagerank[i]*pagerank_weight))
        i+=1
    score_sorted = sorted(allscores,key=lambda x:x[1],reverse = True)[:N]
    return score_sorted

In [15]:

# query ="python"

# tokens = sb.tokenize(query)
# print(tokens)
# #expand_query(tokens)


In [16]:
# res =search_bm25_improved(tokens)
# print(res)

In [17]:
dicti={ "python": [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462],
 "data science": [35458904, 2720954, 54415758, 233488, 12487489, 48364486, 487132, 416589, 27051151, 66414222, 18985040, 4141563, 376707, 67092078, 61624594, 59591015, 49681, 55052729, 31663650, 35757264, 63995662, 3461736, 56499662, 43003632, 26685, 10147369, 24932989, 66982525, 5323, 46207323, 8495, 31915311, 51552534, 64057049, 45443335, 37849704, 42149032, 63198823, 38833779, 26880450, 39171985, 5213, 44133735, 28486111, 48972018, 23943140, 17740009, 49954680, 28326718, 63016369, 56023027, 58731, 55207134, 62798611, 65732625, 58255666, 68181697, 47892671, 1309220, 65499427, 58255600, 47638279, 46626475, 46374359, 53343108, 173332, 7671, 18745015, 50565707, 57143357, 45112545, 63148404, 50533783, 51112472, 49281083, 51505979, 1181008, 56731447, 59116880, 51578025, 21462612, 1765779, 39675445, 42253],
 "migraine": [21035, 36984150, 2702930, 25045060, 24224679, 2555865, 36579642, 310429, 22352499, 11495285, 22294424, 234876, 40748148, 69893, 61962436, 62871079, 843361, 7362700, 16982268, 15712244, 5690287, 7362738, 600236, 12410589, 26584776, 3332410, 20038918, 739855, 1015919, 14201682, 24361010, 53035710, 22901459, 57672434, 4206029, 738384, 36579839, 188521, 15325435, 3602651, 40428462, 322197, 19592340, 3868233, 2385806, 2933438, 23174077, 14001660, 2425344, 288328, 21381229, 26585811, 12652799, 322210, 51078678, 621531, 685130, 11193835, 21197980, 21078348, 3108484, 692988, 31556991, 18741438, 3053003, 50977642, 55115883, 17208913, 64269900, 54077917, 36666029, 50083054, 28245491, 5692662, 18353587, 1994895, 21364162, 20208066, 38574433, 910244, 6154091, 67754025, 2132969, 61386909, 18600765, 579516],
 "chocolate": [7089, 6693851, 6672660, 23159504, 49770662, 167891, 2399742, 100710, 76723, 5290678, 54229, 3881415, 3720007, 32652613, 1471058, 5239060, 1421165, 1730071, 1277918, 7419133, 17720623, 1765026, 19079925, 1979873, 497794, 57947, 15007729, 85655, 4250574, 2386481, 228541, 55225594, 318873, 22631033, 27767967, 7061714, 8175846, 3881265, 3046256, 606737, 845137, 16161419, 3098266, 54573, 11323402, 936243, 39280615, 13687674, 47155656, 7151675, 43627595, 26879832, 43098662, 2333593, 349448, 2052211, 4432842, 56412300, 1411016, 2152015, 3502051, 33372192, 61422444, 2385217, 1217747, 24315397, 7082459, 856246, 6050655, 27162455, 52140446, 37243595, 36961531, 245067, 1148978, 1770825, 976322, 10300434, 7249348, 14945749, 62851606, 637004, 16224368, 18509922],
 "how to make pasta": [25659792, 50404581, 29178, 3080697, 90267, 2568868, 3450096, 49647732, 462173, 43911472, 20229, 40478583, 56643734, 21257512, 2387344, 59405867, 1330188, 12638661, 501757, 446865, 4468939, 25215235, 456363, 95411, 30916, 53487581, 30876926, 301932, 47764842, 426522, 579040, 54155622, 60535326, 23852947, 4275382, 67279077, 16591942, 334546, 602599, 3735620, 10296674, 858120, 30876121, 443480, 1038309, 50653758, 2258995, 34121672, 5382150, 884056, 3141956, 349722, 6745, 3511512, 35211682, 611752, 66963891, 43977806, 36742560, 2899729, 5413930, 61742595, 6972293, 14926, 42674415, 193957, 1950442, 3396753, 2269888, 40055348, 63609800, 1343426, 26078050, 44808, 42444204, 35034191, 9303405, 4627535, 59407816, 28732, 1187122, 6984468, 3328852, 56313776, 43853813, 3533082, 1032674, 39797382, 8892877],
 "Does pasta have preservatives?": [50404581, 301932, 30876121, 12727781, 37018026, 382599, 56232, 1032674, 202437, 9759063, 2047222, 11309920, 15015154, 46482, 406363, 39813131, 9785087, 47840259, 230716, 2240648, 49380722, 62153, 64083, 5355, 1330224, 17345999, 36969, 50577743, 31919750, 1197035, 2175, 15003673, 54145741, 42801, 458008, 47150650, 67922, 39390739, 901091, 28771786, 6984468, 416752, 382619, 198153, 49065540, 496821, 3112548, 15434651, 237489, 47862672, 915309, 708662, 21699434, 550448, 22893145, 17055183, 32863238, 13824676, 1300923, 746225, 33065713, 47770304, 32593, 11002, 20156275, 22735258, 31156754, 32587, 2077960, 344611, 5652480, 31425310, 991758, 40956516, 594987, 1093416, 26473291, 877461, 340356, 13679, 1558639, 11832915, 926863, 14953848, 42155809, 54923, 276975, 66554, 5775715, 3133549, 1104286, 26951370, 3240723, 6523448, 3902658],
 "how google works": [44674524, 1092923, 12431, 224584, 43419164, 9874319, 4338696, 3190097, 9651444, 33321947, 26334893, 1497849, 47489893, 32639051, 2030607, 60903, 19133401, 42411494, 47799755, 4028754, 42960975, 24386465, 5339378, 1494648, 14181749, 58582001, 10062721, 33039125, 286747, 48736239, 33367993, 50575063, 55633178, 29403992, 34113322, 3660182, 25173473, 62438513, 60904, 3235536, 40867519, 5376827, 44424763, 58708106, 64302888, 187946, 29156200, 46426771, 35847782, 5376796, 25295524, 33768164, 46551547, 773423, 5376868, 49931371, 59539691, 22411575, 35326347, 46493906, 11451897, 10619416, 36891093, 40116717, 23533163, 879962, 502593, 3371574, 43194901, 41815118, 35673556, 6575642, 43997189, 5913182, 2126501, 52840911, 1566175, 42951365, 42694174, 22992426, 466299, 736238, 7301470, 1431181, 48653985, 51328172, 12003767],
 "what is information retrieval": [15271, 494530, 442684, 19988623, 731640, 24997830, 10179411, 16635934, 33407925, 11486091, 50716473, 35804330, 18550455, 21106742, 4694434, 26591446, 296950, 24963841, 346470, 509628, 261193, 28688670, 10218640, 1897206, 39000674, 17785794, 38156944, 9586885, 743971, 1185840, 7872152, 10328235, 36794719, 509624, 5818361, 25935906, 22254915, 4881262, 39585214, 30882491, 57312392, 3781784, 25959000, 14109784, 10818378, 25957127, 9511414, 6422823, 20289869, 15101979, 48317971, 14343887, 762092, 4840292, 25130414, 7602386, 37131566, 6118940, 56598843, 11184711, 1315248, 12101316, 22809006, 29979321, 149354, 32817039, 25271852, 20948989, 36749242, 26143506, 19542049, 360030, 20632884, 24963451, 30874683, 11647367, 383162, 13200719, 1981660, 53123104, 10715937, 24019253, 25050663, 27511028, 1514191],
 "NBA": [22093, 16795291, 65166616, 65785063, 835946, 890793, 3921, 450389, 20455, 987153, 240940, 246185, 9000355, 5608488, 3280233, 3505049, 5958023, 72852, 8806795, 1811390, 2423824, 516570, 15392541, 72893, 412214, 278018, 12106552, 42846434, 12754503, 9807715, 4108839, 33328593, 64063961, 7215125, 1811320, 1111137, 5035602, 60483582, 9397801, 255645, 16899, 43376, 72855, 65785040, 72866, 6215230, 4987149, 72878, 16160954, 243389, 64639133, 38958735, 72858, 27196905, 38153033, 1385825, 9733533, 49926096, 4875689, 4750398, 28754077, 43569250, 22092, 72889, 59798759, 49778089, 346029, 8588996, 1956255, 52454088, 25390847, 31667631, 878666, 48695845, 72857, 459304, 27837030, 17107550, 72861, 54859596, 9195892, 6560301, 72875, 72883, 240989, 3196517, 24612090],
 "yoga": [34258, 59580357, 315141, 47035547, 626718, 32016617, 59516988, 61583621, 38424, 196789, 734272, 60851303, 988753, 92222, 419015, 60992198, 621809, 83742, 3127300, 23834216, 1381751, 744158, 18911094, 1817874, 59052395, 43562594, 31793047, 6154795, 17452177, 62342792, 13936750, 60805249, 60823333, 59899027, 46719542, 7702313, 60310807, 44270128, 68489731, 60500106, 1652601, 359983, 60300514, 45482242, 34756533, 58520592, 43611227, 47586513, 1775374, 6933106, 13412771, 62549131, 49278248, 62557135, 4523354, 60468426, 666420, 1017009, 35795560, 34843274, 60199413, 16488467, 29828807, 585681, 41757168, 66018807, 61285582, 1226448, 28848113, 11487904, 4242777, 1661867, 10671559, 44035836, 6931929, 60705564, 61690747, 7343803, 9042644, 36991237, 3965223, 63090179, 632990, 1041815, 53087254, 418999, 1632919],
 "how to not kill plants": [952429, 53916, 4189740, 5132315, 5818788, 35270055, 246674, 28271176, 18952739, 13323241, 1257003, 39562449, 11286, 1440567, 294295, 718110, 5806056, 3587235, 515758, 70129, 11984, 25228985, 1572831, 28125449, 1101651, 8220315, 149463, 63883178, 43937, 432174, 55775421, 2413296, 54019, 1878531, 57559, 1822951, 44568, 10104316, 16671094, 59988, 2068329, 12174150, 18952424, 48340, 83024, 99951, 4390795, 4954263, 3012923, 43344741, 61932070, 1480987, 2520434, 15880608, 7852272, 33084611, 27377596, 57866, 22438472, 8150951, 1265113, 19828134, 76046, 37774, 9270823, 1421709, 2051423, 7173874, 59400, 3606300, 8846, 38980932, 427577, 589513, 147484, 3883838, 29565262, 1527015, 533214, 10669865, 32591, 992052, 4975851, 44629591, 162296, 3513027, 11921706, 9041235, 1068768, 5498413, 591668, 42586177, 1185479, 514894, 3419, 14174833, 71265, 7218230],
 "masks": [63631542, 821829, 633458, 63567907, 14208718, 7912947, 159546, 67759831, 7439323, 64192678, 40909056, 5322079, 63164437, 31728348, 12772, 1402997, 1156703, 1840653, 568164, 1814699, 2945076, 2565663, 442333, 33567480, 468313, 19718702, 987724, 903187, 1485962, 5003908, 4201044, 28086000, 28844729, 1265651, 4659608, 64966775, 55960921, 8970797, 4301719, 35892659, 2248622, 6939163, 48561519, 261396, 34298473, 12263290, 44258772, 25219375, 149426, 6558203, 46567337, 46784964, 6458321, 46576311, 18823362, 48315099, 56440599, 57159776, 74910, 67891964, 1702593, 560306, 1015304, 705756, 7479199, 57772096, 58946599, 1210300, 15716827, 34336876, 9040490, 439102, 264104, 11061915, 39774839, 34735506, 42174581, 66859257, 3293969, 16315, 23166476, 42812440, 66935753, 51990351, 36619752, 59863622, 38404, 64027932],
 "black friday": [366477, 21000624, 3252816, 29437250, 2399020, 2675881, 360797, 184979, 20775117, 3233153, 31185151, 67528700, 31265743, 31526583, 157649, 60421513, 58847654, 360789, 5089316, 37168, 29772915, 29448480, 63351334, 3136619, 8076709, 41047099, 48658625, 50394170, 2621344, 25496814, 2997690, 41956157, 8817935, 19283335, 3738164, 60383204, 29449433, 20302560, 664951, 580517, 21434255, 8304348, 21611071, 55449550, 27555169, 28153175, 39614564, 314544, 31102405, 1428872, 19952476, 59540110, 40704884, 193714, 8948452, 49891754, 24008586, 32047012, 1104969, 54632, 31323977, 28253219, 33538385, 177671, 3286723, 16368206, 25351191, 5549127, 184959, 52708523, 728321, 31186864, 57685061, 240148, 22067678, 20827796, 4681, 41958461],
 "why do men have nipples": [150389, 412643, 48475515, 2775579, 4489, 997303, 250247, 311440, 42669424, 1080173, 2295417, 1243208, 1045845, 1890951, 4668823, 1027896, 7364325, 6613094, 8001012, 45079706, 580715, 5866454, 6383976, 18611260, 28643, 2581217, 5686025, 49998617, 63011479, 54736719, 20913864, 52146943, 34900078, 756765, 344269, 254067, 10916, 1764686, 25211041, 28456222, 35334391, 3845894, 34354998, 403653, 46745914, 11415141, 2252455, 283858, 65217527, 3223727, 42549636, 22546, 581077, 7971671, 21496150, 1104090, 21496038, 12082283, 24385806, 575890, 7105280, 54863336, 36164, 1752731, 3611109, 106121, 50364362, 3039817, 38041, 2968175, 70547, 13502823, 36881878, 702635, 18838, 15179951, 286455, 7139331, 218406, 1130867, 21490868, 1025265, 498971, 12915125, 4819795, 49458922, 23446323, 1244458]}

In [21]:
# for q,l in dicti.items():
#     res = search_bm25_improved(sb.tokenize(q))
#     count=0
#     for a,b in res:
#         if a in l:
#             count+=1
#     print(count/len(l))
    #print(q,count/len(l))
for q,l in dicti.items():
    print (sb.tokenize(q))


['python']
['data', 'science']
['migraine']
['chocolate']
['make', 'pasta']
['pasta', 'preservatives']
['google', 'works']
['information', 'retrieval']
['nba']
['yoga']
['kill', 'plants']
['masks']
['black', 'friday']
['men', 'nipples']


In [10]:
# print(count/len(rel_docs))